Solves the incompressible Navier-Stokes equations in a rectangular domain with prescribed velocities along the boundary. The solution method is finite differencing on a staggered grid with implicit diffusion and a Chorin projection method for the pressure. Visualization is done by a colormap-isoline plot for pressure and normalized quiver and streamline plot for the velocity field. The standard setup solves a lid driven cavity problem.

Convert to Python according to this paper https://math.mit.edu/~gs/cse/codes/mit18086_navierstokes.pdf

In [1]:
import numpy as np
import scipy 
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.sparse import diags, kron, csr_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee


import tqdm


In [2]:
def avg(A, k=1) -> np.ndarray:
    """
    Average adjacent elements in array A over k iterations.
    
    Args:
    A (np.ndarray): The input array.
    k (int): Number of iterations to average over.
    
    Returns:
    np.ndarray: Array after averaging adjacent elements k times.
    """
    A = np.asarray(A)
    if A.ndim == 1:
        A = A.reshape(-1, 1)  # Make A a column vector if it is a row vector
    
    B = A
    for _ in range(k):
        B = (B[:-1] + B[1:]) / 2
    
    if A.shape[0] == 1:
        B = B.T  # Transpose back to row vector if A was originally a row vector
    
    return B

# Example usage
A = np.array([1, 2, 3, 4, 5])
result = avg(A, k=2)
print(result)


[[2.]
 [3.]
 [4.]]


In [3]:
def lap1d(n, h, a11) -> np.ndarray:
    e = np.ones(n)
    data = np.array([-e, 2 * e, -e])
    offsets = np.array([-1, 0, 1])
    lap = sp.diags(data, offsets, shape=(n, n)) / (h ** 2)
    if a11 == 1:
        lap = lap.tolil()
        lap[0, 0] = 1
        lap[-1, -1] = 1
        lap = lap.tocsc()
    return lap

# Example usage:
n = 5  # number of points
h = 1.0  # spacing
a11 = 2  # Dirichlet boundary condition
A = lap1d(n, h, a11)
print(A.toarray())  # Convert to a dense matrix for display


[[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]


In [52]:
def symmetric_amd_permutation(A) -> np.ndarray:
    """
    Compute the symmetric approximate minimum degree permutation of a sparse matrix.

    Parameters:
    A (scipy.sparse.csr_matrix): The input sparse matrix.

    Returns:
    numpy.ndarray: The permutation vector.
    """
    # Ensure the matrix is in CSR format
    if not isinstance(A, csr_matrix):
        A = csr_matrix(A)
    
    # Compute the reverse Cuthill-McKee ordering
    perm = reverse_cuthill_mckee(A, symmetric_mode=True)
    perm = np.array(perm, dtype = int)
    
    return perm

# Example usage
if __name__ == "__main__":
    # Create an example sparse matrix
    n = 5
    A = csr_matrix([
        [10, 1, 0, 0, 2],
        [1, 9, 0, 0, 0],
        [0, 0, 7, 3, 0],
        [0, 0, 3, 8, 4],
        [2, 0, 0, 4, 6]
    ])
    
    # Compute the symmetric AMD permutation
    perm = symmetric_amd_permutation(A)
    
    # Print the permutation
    print("Permutation vector:", perm)


Permutation vector: [2 3 4 0 1]


In [5]:
#Define parameter

Re = 3000    # Reynold number
dt = 0.01    # time-step
tf = 3       # final time
lx = 1       # width of the box
ly = 1       # height of the box
nx = 100     # number of x-gridpoints
ny = 111     # number of y-gridpoints 
nsteps = 30  # number of steps with graphic output

In [6]:
#Set up the grid
nt = np.ceil(tf/dt)  # Round down to the nearest integer aka #of iters
x = np.linspace(start = 0, stop = lx, num = nx + 1)  # Vector of x from 0 to lx = 1, and the step size is nx + 1 = 101
hx = lx/nx           # Spatial discretization step sizes in x direction i.e lx/nx = 0.01
y = np.linspace(start = 0, stop = ly, num = ny + 1)  # Vector of y from 0 to ly = 1 and the step size is ny + 1 = 112
hy = ly/ny           # Spatial discretization step sizes in y direction i.e ly/ny
X, Y = np.meshgrid(x, y)  # Generate the meshgrid

In [7]:
#Inital condition

U = np.zeros((nx - 1, ny))
V = np.zeros((nx, ny - 1))

In [8]:
#Boundary condition

uN = x*0 + 1      #u - north
vN = avg(x)*0     #v - north
uS = x*0          #u - south
vS = avg(x)*0     #v - south
uW = avg(y)*0     #u - west
vW = y*0          #v - west
uE = avg(y)*0     #u - east
vE = y*0          #v - east

In [9]:
#U with boundary condition

Ubc = dt/Re *((np.hstack([2* uS[1:-1].reshape(1, -1).T, np.zeros((nx - 1, ny - 2)), 2* uN[1:-1].reshape(1,-1).T]) / hx ** 2) + (np.vstack([uW.T, np.zeros((nx - 3, ny)), uE.T]) / hy ** 2))


In [10]:
#V with boundary condition

Vbc = dt/Re * ((np.hstack([vS, np.zeros((nx, ny - 3)), vN]) / hx ** 2) + (np.vstack([2* vW[1:-1].reshape(1, -1), np.zeros((nx - 2, ny -1)), 2* vE[1:-1].reshape(1,-1)]) / hy ** 2))

In [43]:
#Diffusion term in the u-velocity

Lu = np.eye((nx - 1)* ny) + dt/Re * (kron(np.eye(ny), lap1d(nx-1, hx, 2)) + kron(lap1d(ny, hy, 3), np.eye(nx - 1)))
peru = symmetric_amd_permutation(Lu)
Lu_perm = Lu[peru, :][:, peru]
Ru = np.linalg.cholesky(Lu_perm)
Ru_transpose = Ru.T


In [41]:
#Diffusion term in the v-velocity

Lv = np.eye(nx* (ny - 1)) + dt/Re * (kron(np.eye(ny - 1), lap1d(nx, hx, 3)) + kron(lap1d(ny - 1, hy, 2), np.eye(nx)))
perv = symmetric_amd_permutation(Lv)
Lv_perm = Lv[perv, :][:, perv]
Rv = np.linalg.cholesky(Lv_perm)
Rv_transpose = Rv.T

In [42]:
Rv.shape    

(11000, 11000)

In [45]:
#For the stream function

Lq = kron(np.eye(ny - 1), lap1d(nx - 1, hx, 2)) + kron(lap1d(ny - 1, hy, 2), np.eye(nx - 1))
perq = symmetric_amd_permutation(Lq)
Lq_perm = Lq[perq, :][:, perq]
# Rq = np.linalg.cholesky(Lq_perm)
# Rq_transpose = Rq.T

In [59]:
np.linalg.cholesky(Lq_perm)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

: 

In [17]:
Lq_perm.shape, Lv_perm.shape    

((10890, 10890), (11000, 11000))

In [13]:
#Pressure correction for pressure

Lp = kron(np.eye(ny), lap1d(nx, hx, 1)) + kron(lap1d(ny, hy, 1), np.eye(nx)) 

Lp = Lp.tolil()
Lp[0, 0] = 3/2 * Lp[0, 0]
Lp = Lp.tocsc()

perp = spla.splu(Lp).perm_r
Lp_perp = Lp[perp, :][:, perp]

Rp = np.linalg.cholesky(Lp_perp)
Rp_transpose = Rp.T


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [23]:
perp = spla.splu(Lp).perm_r


In [18]:
Lp[0, 0] = 3/2 * Lp[0, 0]
print(Lp[0, 0])

33481.5


In [16]:
Lp_perm.shape

(11100, 11100)

In [14]:
perp = symmetric_amd_permutation(Lp)
Lp_perm = Lp[perp, :][:, perp]
Rp = np.linalg.cholesky(Lp_perm)
Rp_transpose = Rp.T

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [49]:
# Treat nonlinear term

gamma = np.minimum(1.2 * dt * np.maximum(np.max(abs(U)), np.max(abs(V))), 1).reshape(1)
 

#Add B.C to u - velocity
Ue = np.vstack([uW.T, U, uE.T]) #shape (101, 111)
Ue = np.hstack([(2 * uS.reshape(1,-1) - Ue[:,1].reshape(1,-1)).T, Ue, (2 * uN.reshape(1,-1) - Ue[:,-1]).T]) #shape (101, 113)

#Add B.C to v - velocity
Ve = np.hstack([vS, V, vN]) #(100, 112)
Ve = np.vstack([(2 * vW.reshape(1,-1) - Ve[1,:]), Ve, (2 * vE - Ve[-1, :])]) #shape (102, 112)

#Average and apply finite difference to u - velocity
Ua = avg(Ue.T).T
Ud = np.diff(Ue, n = 1) / 2

#Average and apply finite difference to v - velocity
Va = avg(Ve) 
Vd = np.diff(Ve.T, n = 1) / 2 

#Finite difference the nonlinear tearm (uv)_x and (uv)_y
UVx = np.diff((Ua * Va - gamma * abs(Ua) * Vd.T).T).T / hx # (uv)_x, shape (100, 112)
UVy = np.diff(Ua * Va - gamma * Ud * abs(Va)) / hy     # (uv)_y, shape (101, 111)

#Update the average and approximate the derivative
Ua = avg(Ue[:,1:-1]) #shape (100, 111)
Ud = np.diff(Ue[:,1:-1].T).T / 2 #shape (100, 111)
Va = avg(Ve[1:-1].T).T #shape (100, 111)
Vd = np.diff(Ve[1:-1,:]) / 2 #shape (100, 111)

#Compute (u^2)_x and (v^2)_y by averaging u - horizontally and v - vertically
U2x = np.diff((Ua ** 2 - gamma * abs(Ua) * Ud).T).T / hx  #shape (99, 111)
V2y = np.diff(Va ** 2 - gamma * abs(Va) * Vd) / hy        #shape (100, 110)

#Update values of interior points
u = U - dt * (UVy[1:-1, :] + U2x) #shape (99, 111)
v = V - dt * (UVx[:, 1:-1] + V2y) #shape (100, 110)

TypeError: 'module' object is not callable

In [38]:
#Implicit velocity

#u* intermediate velocity
rhs = (U + Ubc).flatten()
rhs_permuted = rhs[peru]
y = spla.spsolve_triangular(Ru_transpose, rhs_permuted, lower = True)
u = spla.spsolve_triangular(Ru, y, lower = False)
u_original = np.empty_like(u)
u_original[peru] = u
u_reshaped = np.reshape(u_original, (nx - 1, ny))




C:\Users\hoang\AppData\Local\Temp\ipykernel_11796\3169962199.py:6: SparseEfficiencyWarning: CSR matrix format is required. Converting to CSR matrix.
  y = spla.spsolve_triangular(Ru_transpose, rhs_permuted, lower = True)


LinAlgError: A is not triangular: A[0, 2] is nonzero.

In [37]:
rhs_permuted.shape

(10989,)

In [30]:
u.shape 

(99, 111)

In [371]:
u.shape, v.shape    

((99, 111), (100, 110))

In [356]:
UVx.shape, UVy.shape, U2x.shape, V2y.shape    

((100, 112), (101, 111), (99, 111), (100, 110))

In [319]:
Ua.shape, Ud.shape, Va.shape, Vd.shape    

((100, 111), (100, 111), (100, 111), (100, 111))